In [ ]:
import os
import re
import sys
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
if '../scripts' not in sys.path:
    sys.path.append('../scripts')
# from plots import plot_wmx, plot_wmx_avg, plot_w_distr, plot_weights, save_selected_w
%matplotlib inline

In [ ]:
athorny_PCs_ratio = 0.0                                                                            
data = np.load(f'../files/spike_times_a-thorny_place_cells_ratio={athorny_PCs_ratio:.1f}.npz',
               allow_pickle=True)
spike_times = data['spike_trains'].item()
PCs = data['place_cell'].item()
cell_types = PCs.keys()
selected_PCs = {}
selected_not_PCs = {}
n_selected_PCs, n_selected_not_PCs = 5, 5
for cell_type in cell_types:
    idx, = np.where(PCs[cell_type] == True)
    if len(idx) > 0:
        jdx = np.linspace(0, idx.size-1, n_selected_PCs+2, dtype=np.int32)
        selected_PCs[cell_type] = idx[jdx[1:-1]]
    else:
        selected_PCs[cell_type] = np.random.permutation(PCs[cell_type].size)[:n_selected_PCs]
        selected_PCs[cell_type].sort()
    idx, = np.where(PCs[cell_type] == False)
    selected_not_PCs[cell_type] = np.random.permutation(PCs[cell_type].size)[:n_selected_not_PCs]
    selected_not_PCs[cell_type].sort()

In [ ]:
data = np.load(f'../files/weights_a-thorny_place_cells_ratio={athorny_PCs_ratio:.1f}.npz',
               allow_pickle=True)
weights = data['weights'].item()
config = data['config'].item()
connections = config['connectivity']

In [ ]:
pre, post = 'thorny', 'thorny'
W = weights[pre][post].toarray()
cell_id = selected_PCs[post][2]
# cell_id = selected_not_PCs[post][2]
wgts = W[:,cell_id]

In [ ]:
PC, = np.where((PCs[pre] == True) & (wgts > 0))
not_PC, = np.where((PCs[pre] == False) & (wgts > 0))
pos_PC_wgts = PC.size / np.sum(PCs[pre]) * 100
pos_not_PC_wgts = not_PC.size / np.sum(np.logical_not(PCs[pre])) * 100
print(f'{pos_PC_wgts:.2f}% of weights from place cells are > 0.')
print(f'{pos_not_PC_wgts:.2f}% of weights from non place cells are > 0.')

ms = 4
fig,ax = plt.subplots(1, 1, figsize=(6,4))
PC_col = [.7, 0, .7]
not_PC_col = [0, .7, 0]
ax.plot(not_PC, wgts[not_PC] * 1e9, '.', color=not_PC_col, markersize=ms, label='Not PCs')
ax.plot(PC, wgts[PC] * 1e9, 'o', color=PC_col, markerfacecolor=PC_col, markersize=ms, label='PCs')
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.set_xlabel('Presynaptic neuron #')
ax.set_ylabel('Synaptic weight (nS)')
ax.legend(loc='best')
fig.tight_layout()                                                                                                                                                                                  